In [2]:
import os
os.getcwd()

'/home/jovyan/ArditArifi'

In [3]:
source_path="/home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6"
output_path="/home/jovyan/student-storages/GROUP3/ArditArifi/output"

! mkdir -p output_path

Analysis

In [4]:
import glob
# ANALYSIS
import xarray as xr
import pandas as p
import numpy as np
# PLOTTING
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as colors


In [29]:
import math
def compute_cell_areas(ds, lat_dim="lat", lon_dim="lon", R=6371000.0):

    # Extract 1D lat, lon (cell centers)
    lats = ds[lat_dim]
    lons = ds[lon_dim]

    # Earth radius in meters
    R = 6371000.0
    
    # Copy and adjust longitudes so they range in [-180..180]
    lons_ = np.copy(lons)
    lons_[lons_ > 180] = lons_[lons_ > 180] - 360.0
    
    # Prepare array for surface area
    surf = np.zeros((len(lats), len(lons)))

    # Inline function to compute distance using haversine formula
    def spherical_distance(lat1, lon1, lat2, lon2):
        # Convert degrees to radians
        lat1r = math.radians(lat1)
        lon1r = math.radians(lon1)
        lat2r = math.radians(lat2)
        lon2r = math.radians(lon2)
        
        dlat = lat2r - lat1r
        dlon = lon2r - lon1r
        
        # Haversine formula
        a = (math.sin(dlat / 2))**2 + math.cos(lat1r) * math.cos(lat2r) * (math.sin(dlon / 2))**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        
        return R * c  # distance in meters

    # Loop through each cell
    # We'll assume lats[k+1] and lons[l+1] exist, so go up to len(...) - 1
    for k in range(len(lats) - 1):
        for l in range(len(lons) - 1):
            # Coordinates for cell corners
            # dx = distance between (lats[k], lons_[l]) and (lats[k], lons_[l+1])
            # dy = distance between (lats[k], lons_[l]) and (lats[k+1], lons_[l])
            
            dx = spherical_distance(lats[k], lons_[l], lats[k], lons_[l+1])
            dy = spherical_distance(lats[k], lons_[l], lats[k+1], lons_[l])
            
            # Area approx = dx * dy
            surf[k, l] = dx * dy

    # Normalize so the largest cell = 1
    surf_max = np.max(surf)
    if surf_max > 0:
        surf /= surf_max

    # Optional patching near the poles:
    if np.max(lats) < 0:
        # Entire domain is in the Southern Hemisphere
        surf[-1, :] = 1
    if np.min(lats) > 0:
        # Entire domain is in the Northern Hemisphere
        valid = surf[surf > 0]
        if len(valid) > 0:
            surf[-1, :] = np.min(valid)
    surf = xr.DataArray(
        data=surf,
        coords={lat_dim: lats, lon_dim: lons},
        dims=[lat_dim, lon_dim],
        name="cell_area"
    )
    return surf


In [30]:

# REGIONS
region_label = ["ARC", "ANT"]
arctic_lat = [60, 90]
antarctic_lat = [-90, -60]

# SEASONS
seasons_label = ["DJF", "MAM", "JJA", "SON"]
seasons = {
    "DJF": [12, 1, 2],   # Dec, Jan, Feb
    "MAM": [3, 4, 5],    # Mar, Apr, May
    "JJA": [6, 7, 8],    # Jun, Jul, Aug
    "SON": [9, 10, 11],  # Sep, Oct, Nov
}

# Create combined region-season labels
region_season_label = [f"{region} {season}" for region in region_label for season in seasons_label]

# EXPERIMENTS
experiments = ["control", "2xss", "2xdust", "2xfire", "2xDMS"]

# MODELS
models = ["IPSL-CM6A-LR-INCA", "UKESM1-0-LL", "NorESM2-LM"]

# PATTERN FUNCTION
patter = lambda model, var, exp: f"{source_path}/{var}*{model}*piClim-{exp}*.nc"

In [32]:
# CONSTRUCT TABLE
mean_values = np.full(
    shape=(len(region_season_label), len(experiments), len(models)),
    fill_value=np.nan
)
std_values = np.full(
    shape=(len(region_season_label), len(experiments), len(models)),
    fill_value=np.nan
)


# LOAD DATA
model = ""
var = "rtmt"

# Calculate total steps for progress
total_steps = len(models) * len(experiments) * len(seasons_label)
current_step = 0

# MAIN LOOP
for idx_model, model in enumerate(models):
    for idx_exp, exp in enumerate(experiments):
        ds = None
        try:
            # Attempt to open the dataset
            ds = xr.open_mfdataset(patter(model, var, exp), combine="by_coords")
            ds = ds[var]
            # 1) Load dataset for this model/var/exp
        except Exception as e:
            # If there's an error reading files, set to 0 for ARC and ANT
            print(f"Error opening files {patter(model, var, exp)}: {e}")
            current_step += len(seasons_label)
            continue               

        weights = compute_cell_areas(ds)

        for idx_season, season in enumerate(seasons_label):
            # Increment the progress
            current_step += 1
            # Compute and display percentage
            pct = (current_step / total_steps) * 100
            print(f"Progress: {current_step}/{total_steps} -> {pct:.2f}%")
            
            # Select months for this season
            ds_season = ds.sel(time=ds.time.dt.month.isin(seasons[season]))

            # ---------------------
            # ARCTIC
            # ---------------------
            ds_season_arctic = ds_season.sel(
                lat=slice(arctic_lat[0], arctic_lat[1]),
                lon=slice(-180, 180)
            )
            weights_arctic = weights.sel(
                lat=slice(arctic_lat[0], arctic_lat[1]),
                lon=slice(-180, 180)
            )
            
            ds_season_arctic_area_ts = ds_season_arctic.weighted(weights_arctic).mean(dim=("lat", "lon"))
            
            arc_mean = ds_season_arctic_area_ts.mean("time").values
            mean_values[idx_season, idx_exp, idx_model] = arc_mean
            
            arc_std = ds_season_arctic_area_ts.std("time").values
            std_values[idx_season, idx_exp, idx_model] = arc_std
            
            # ---------------------
            # ANTARCTIC
            # ---------------------
            ds_season_antarctic = ds_season.sel(
                lat=slice(antarctic_lat[0], antarctic_lat[1]),
                lon=slice(-180, 180)
            )
            weights_antarctic = weights.sel(
                lat=slice(antarctic_lat[0], antarctic_lat[1]),
                lon=slice(-180, 180)
            )
            
            ds_season_antarctic_area_ts = ds_season_antarctic.weighted(weights_antarctic).mean(dim=("lat", "lon"))
            
            ant_mean = ds_season_antarctic_area_ts.mean("time").values
            mean_values[idx_season + 4, idx_exp, idx_model] = ant_mean
            print(ant_mean)
            ant_std = ds_season_antarctic_area_ts.std("time").values
            std_values[idx_season + 4, idx_exp, idx_model] = ant_std
            
            ds_season_antarctic_area_ts = ds_season_antarctic.weighted(weights_antarctic).mean(dim=("lat", "lon"))
            
            ant_mean = ds_season_antarctic_area_ts.mean("time").values
            mean_values[idx_season + 4, idx_exp, idx_model] = ant_mean
            
            ant_std = ds_season_antarctic_area_ts.std("time").values
            std_values[idx_season + 4, idx_exp, idx_model] = ant_std
            
# Finally, display the populated array
print("Shape of mean_values:", mean_values.shape)
print(mean_values)

Progress: 1/60 -> 1.67%
-8.69911013908407
Progress: 2/60 -> 3.33%
-134.42781010121266
Progress: 3/60 -> 5.00%
-148.10770221034443
Progress: 4/60 -> 6.67%
-76.55876348484898
Progress: 5/60 -> 8.33%
-9.33651795352111
Progress: 6/60 -> 10.00%
-134.59254705072541
Progress: 7/60 -> 11.67%
-148.1723510253973
Progress: 8/60 -> 13.33%
-76.63888112996656
Progress: 9/60 -> 15.00%
-8.99184254409099
Progress: 10/60 -> 16.67%
-134.39725009375164
Progress: 11/60 -> 18.33%
-148.35722590484886
Progress: 12/60 -> 20.00%
-76.630583927784
Error opening files /home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6/rtmt*IPSL-CM6A-LR-INCA*piClim-2xfire*.nc: no files to open
Error opening files /home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6/rtmt*IPSL-CM6A-LR-INCA*piClim-2xDMS*.nc: no files to open
Progress: 21/60 -> 35.00%
-13.36197706573498
Progress: 22/60 -> 36.67%
-130.89670233538132
Progress: 23/60 -> 38.33%
-144.3388072720646
Progress: 24/60 -> 40.00%
-73.85044568341033
Progress: 25/60 -> 41.67%
-13.761636

In [10]:
import pandas as pd

bias = mean_values[:, 1:, :] - mean_values[:, 0, :][:, np.newaxis, :]
for idx_exp in range(len(experiments) - 1):
    # This will skip the control experiment at index 0
    exp_name = experiments[1 + idx_exp]
    print("Experiment:", exp_name)
    
    # bias[:, idx_exp, :].shape -> (len(region_season_label), len(models))
    # bias[:, idx_exp, :].transpose().shape -> (len(models), len(region_season_label))
    # We want models as rows and region_season_label as columns
    df = pd.DataFrame(
        data=bias[:, idx_exp, :].transpose(),
        index=models,
        columns=region_season_label
    )

    print(df)
    print()  # Blank line for readability

Experiment: 2xss
                    ARC DJF   ARC MAM   ARC JJA   ARC SON   ANT DJF   ANT MAM  \
IPSL-CM6A-LR-INCA  0.157356 -0.600879 -1.143466 -0.077902 -0.637407 -0.164734   
UKESM1-0-LL       -0.260221  0.168703 -0.497273 -0.050565 -0.399657 -0.295128   
NorESM2-LM        -0.802078 -0.343978 -0.872966 -0.441364 -1.438295 -0.408204   

                    ANT JJA   ANT SON  
IPSL-CM6A-LR-INCA -0.064649 -0.080119  
UKESM1-0-LL        0.103706 -0.346260  
NorESM2-LM        -0.474610 -0.778683  

Experiment: 2xdust
                    ARC DJF   ARC MAM   ARC JJA   ARC SON   ANT DJF   ANT MAM  \
IPSL-CM6A-LR-INCA  0.557546 -0.596717 -0.590374  0.374631 -0.292733  0.030561   
UKESM1-0-LL       -0.096941  0.280963 -0.422601  0.101109 -0.039652 -0.178474   
NorESM2-LM         0.089839  0.680931  0.548995  0.042616 -0.334020  0.565876   

                    ANT JJA   ANT SON  
IPSL-CM6A-LR-INCA -0.249524 -0.071822  
UKESM1-0-LL        0.129572  0.117032  
NorESM2-LM         1.228110  0.92

In [23]:
import pandas as pd
import numpy as np

# Suppose the below variables are already defined
# bias : shape (len(region_season_label), len(experiments)-1, len(models))
# region_season_label : list of region/season names
# experiments : list of experiments, with experiment[0] = "control"
# models : list of model names

all_data = []  # Will hold the melted DataFrame for each experiment

for idx_exp in range(len(experiments) - 1):
    exp_name = experiments[idx_exp + 1]
    print("Experiment:", exp_name)

    # bias[:, idx_exp, :].shape -> (n_region_season, n_models)
    # transpose -> (n_models, n_region_season)
    df = pd.DataFrame(
        data=bias[:, idx_exp, :].transpose(),
        index=models,
        columns=region_season_label
    )

    # Reshape df to "long" format with columns [Model, region_season_label, value]
    df_long = df.reset_index().melt(
        id_vars="index",                 # "index" column is the model
        var_name="RegionSeason",         # name for the melted columns
        value_name="Bias"               # name for the melted values
    )

    # Rename "index" -> "Model"
    df_long.rename(columns={"index": "Model"}, inplace=True)

    # Add a column for Experiment
    df_long["Experiment"] = exp_name

    # Append to the list
    all_data.append(df_long)

# Concatenate all experiments into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Optionally reorder columns (Experiment, Model, RegionSeason, Bias)
final_df = final_df[["Experiment", "Model", "RegionSeason", "Bias"]]

# Save to CSV
final_df.to_csv("all_bias_results.csv", index=False)

print("CSV file saved: all_bias_results.csv")


Experiment: 2xss
Experiment: 2xdust
Experiment: 2xfire
Experiment: 2xDMS
CSV file saved: all_bias_results.csv
